In [1]:
import numpy as np
import pandas as pd
import calendar
from os.path import join
import json
from urllib.parse import quote
from urllib.request import urlopen

In [2]:
# Load datasets from csvs
train_raw = pd.read_csv(join('data', 'train.csv'),
                    usecols=['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'NumMosquitos', 'WnvPresent'],
                    dtype={'Trap': 'category'},
                    parse_dates=['Date'])

weather_raw = pd.read_csv(join('data', 'weather.csv'),
                      parse_dates=['Date'])

In [3]:
# Water1 = 0 for all observations, drop it
weather = weather_raw.drop(['Water1'], axis=1)
# 'T' stands for 'trace amount' replace with 0 
weather.replace('  T', 0, inplace=True)

# Convert rows that are interpreted as object to numeric
def numeric(x):
    if x.dtype=='object' and x.name != 'CodeSum':
        return pd.to_numeric(x, errors='coerce')
    else:
        return x

weather = weather.apply(numeric)

# '-' represents a missing value
weather.replace('-', np.NaN, inplace=True)

# Replace the ResultDir, which is in degress, to sin and cos
# which are cyclical
weather['ResultDirSin'] = np.sin(np.radians(weather['ResultDir']))
weather['ResultDirCos'] = np.cos(np.radians(weather['ResultDir']))
weather.drop('ResultDir', axis=1, inplace=True)

# 'TS', Thunderstorm, and 'FG' fog, are high moisture events
# they might be useful. Drop other flags
def findCodeSum(x, code):
    if x.CodeSum.find(code) != -1:
        return 1
    else:
        return 0
    
weather['TS'] = weather.apply(findCodeSum, axis=1, code='TS', reduce=False)
weather['FG'] = weather.apply(findCodeSum, axis=1, code='FG', reduce=False)

weather.drop('CodeSum', axis=1, inplace=True)

In [4]:
# Separate weather by station
weather_stn1 = weather[weather['Station'] == 1]
weather_stn2 = weather[weather['Station'] == 2]
weather_stn1 = weather_stn1.drop('Station', axis=1).reset_index(drop=True)
weather_stn2 = weather_stn2.drop('Station', axis=1).reset_index(drop=True)

# Convert sunrise and sunset times to minutes past midnight
weather_stn1['Sunrise'] = weather_stn1.Sunrise.apply(
    lambda x: int(str(x)[:-4]) * 60 + int(str(x)[-4:-2]))
weather_stn1['Sunset'] = weather_stn1.Sunset.apply(
    lambda x: int(str(x)[:-4]) * 60 + int(str(x)[-4:-2]))

In [5]:
# Create 20 days preceding averages of some features
to_mean = ['Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 
           'Heat', 'Cool', 'Depth', 'SnowFall', 'PrecipTotal', 'ResultSpeed',
           'ResultDirSin', 'ResultDirCos', 'AvgSpeed']

for feature in to_mean:
    
    tmp = []
    for i in range(len(weather_stn1)):
        tmp.append(np.mean(weather_stn1[feature].iloc[max(0, i-20):i+1]))
    weather_stn1[feature+'Mean'] = tmp
    
    tmp = []
    for i in range(len(weather_stn2)):
        tmp.append(np.mean(weather_stn2[feature].iloc[max(0, i-20):i+1]))
    weather_stn2[feature+'Mean'] = tmp

In [6]:
# Calculate days since the last rain
days = 0
DaysSinceRain = []

for row in weather_stn1.itertuples():
    DaysSinceRain.append(days)
    if row.PrecipTotal == 0:
        days += 1
    else:
        days = 0
        
weather_stn1['DaysSinceRain'] = DaysSinceRain

days = 0
DaysSinceRain = []

for row in weather_stn2.itertuples():
    DaysSinceRain.append(days)
    if row.PrecipTotal == 0:
        days += 1
    else:
        days = 0
        
weather_stn2['DaysSinceRain'] = DaysSinceRain

In [7]:
# Find rotation matrix that places the two weather stations
# on a single axis
p1 = np.array([-87.933, 41.995])
p2 = np.array([-87.752, 41.786])
p = p1 - p2

angle = np.arctan((p[0])/(p[1]))
rot_mat = [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]

p_dist = np.matmul(rot_mat, p)[1]

# This function will take a date and trap location,
# rotate the map so that the stations are one one axis
# then calculate the linear interpolation of the weather
# at the two stations, and assign the corresponding weather data
# at the y coordinate of the trap
def weather_gen(date, lat, lon):
    
    return_data = pd.DataFrame()

    x = np.array([lon, lat]) - p2
    x_dist = np.matmul(rot_mat, x)[1]
    
    w1 = weather_stn1[weather_stn1.Date==date]
    w2 = weather_stn2[weather_stn2.Date==date]
    
    if len(w1) != 1 or len(w2) != 1:
        raise ValueError('Too many values for single date')
    
    for col in weather_stn1.columns[1:]:
        if col in ['DaysSinceRain', 'TS', 'FG']:
            if x_dist > p_dist:
                return_data[col] = w1[col].values
            else:
                return_data[col] = w2[col].values
        elif any(pd.isnull(w1[col])):
            return_data[col] = w2[col].values
        elif any(pd.isnull(w2[col])):
            return_data[col] = w1[col].values
        else:
            return_data[col] = [np.interp(x_dist, [0, p_dist], np.append(w2[col], w1[col]))]
    
    return return_data

In [8]:
# Find all unique date and location pairs in the dataset
weather_combos = train_raw[['Date', 'Latitude', 'Longitude']].groupby(
    ['Date', 'Latitude', 'Longitude']).count().reset_index()

# Calculate the weather for all the combos
weather_tmp = pd.DataFrame(columns=weather.columns[2:])

for row in weather_combos.itertuples():
    weather_tmp = weather_tmp.append(
        weather_gen(row.Date, row.Latitude, row.Longitude), ignore_index=True)

weather_data = pd.concat([weather_combos, weather_tmp], axis=1)

# Make sure that some features don't get converted to object
weather_data['TS'] = weather_data.TS.astype(int)
weather_data['FG'] = weather_data.FG.astype(int)

weather_data['Sunrise'] = weather_data.Sunrise.astype(int)
weather_data['Sunset'] = weather_data.Sunset.astype(int)

In [9]:
# This will calculate the zone of a location
# data is acquired from Chicago Data Portal
# using their SODA api, return 0 if there is no
# data for the location
def get_zone(lat, lon):
    query = ("https://data.cityofchicago.org/resource/dj47-wfun.geojson?"+
             "$$app_token=DktAvFbM8ptYxalNeI1J4OvgP&$where=intersects(the_geom,"+
             quote(f" 'POINT ({lon} {lat})')"))
    response = urlopen(query)
    data = json.loads(response.read().decode('utf-8'))
    
    if not data['features']:
        return 0
    else:
        return int(data['features'][0]['properties']['zone_type'])

In [10]:
# Find zone for all traps
trap_zones = []

traps = train_raw[['Trap', 'Latitude', 'Longitude']]

for trap in traps.Trap.astype('category').cat.categories:
    trap_zones.append([
        trap, get_zone(traps[traps.Trap == trap].iloc[0].Latitude,
                       traps[traps.Trap == trap].iloc[0].Longitude)
    ])

trap_zones = pd.DataFrame(trap_zones, columns=['Trap', 'Zone'])

In [11]:
# Count how many times a trap is split for a certain date, trap, and species
train_trap_count = train_raw.groupby(
    ['Date', 'Trap', 'Species']).size().reset_index().rename(
        columns={0: 'TrapCount'})

In [12]:
# These three species are the only ones that have been detected
# with the virus
valid_species = ['CULEX PIPIENS/RESTUANS', 'CULEX PIPIENS','CULEX RESTUANS']

In [13]:
# Code the species data into one hot, only keeping the valid species
train_species = pd.get_dummies(train_raw.Species, prefix='', prefix_sep='')
train_species = train_species[valid_species]

train = train_raw.join(train_species)

In [14]:
# Merge all mosquito, calculated weather, and trap zone information
train = train.merge(
    train_trap_count, on=['Date', 'Trap', 'Species'], how='left').merge(
        trap_zones, on=['Trap'], how='left').merge(
            weather_data, on=['Date', 'Latitude', 'Longitude'], how='left')

In [15]:
# Code zone into one hot variables
train_zone_one_hot = pd.get_dummies(train.Zone, prefix='Zone', prefix_sep='', drop_first=True)

train = train.join(train_zone_one_hot)

In [16]:
# Code month into one hot
train_month_one_hot = pd.get_dummies(
    train.Date.apply(lambda x: x.month),
    prefix='',
    prefix_sep='',
    drop_first=True)
train_month_one_hot.columns = [
    calendar.month_abbr[int(x)] for x in train_month_one_hot.columns
]

train = train.join(train_month_one_hot)

In [17]:
# Code year into one hot
train_year_one_hot = pd.get_dummies(
    train.Date.apply(lambda x: x.year),
    prefix='',
    prefix_sep='',
    drop_first=True)

train = train.join(train_year_one_hot)

In [18]:
# Save new datasets
pd.to_pickle(train.pickle, 'train')

pd.to_pickle(trap_zones.pickle, 'trap_zones')